In [ ]:
# ====================================================
# 0. 환경 설정 (필요 패키지 설치)
# ====================================================
!pip install -U accelerate peft bitsandbytes
!pip install huggingface_hub

In [ ]:
# transformers repo clone 및 설치
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install -e .

In [ ]:
from huggingface_hub import login, HfApi, upload_file
login()

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ====================================================
# 1. 사용자 설정
# ====================================================
model_list = [
    {
        "model_id": "LeannaJ/gemma3n-lora-summary",
        "repo_id_gguf": "LeannaJ/gemma3n-lora-gguf-summary",
        "local_model_path": "/content/drive/MyDrive/Gemma_FineTuning/GGUF/gguf_input_summary",
        "gguf_output_dir": "/content/drive/MyDrive/Gemma_FineTuning/GGUF/gguf_output_summary"
    },
    {
        "model_id": "LeannaJ/gemma3n-lora-math",
        "repo_id_gguf": "LeannaJ/gemma3n-lora-gguf-math",
        "local_model_path": "/content/drive/MyDrive/Gemma_FineTuning/GGUF/gguf_input_math",
        "gguf_output_dir": "/content/drive/MyDrive/Gemma_FineTuning/GGUF/gguf_output_math"
    }
]

quant_method = "q4_0"  # 또는 "f16", "q8_0"

In [ ]:
# ====================================================
# 2. GGUF 변환 실행 (transformers/scripts/convert/gguf/convert.py)
# ====================================================
from transformers import AutoTokenizer, AutoModelForCausalLM
import os, glob

api = HfApi()

for m in model_list:
    print(f"\n🚀 변환 시작: {m['model_id']}")

    # 디렉토리 준비
    os.makedirs(m["local_model_path"], exist_ok=True)
    os.makedirs(m["gguf_output_dir"], exist_ok=True)

    # 모델 다운로드 및 저장
    tokenizer = AutoTokenizer.from_pretrained(m["model_id"])
    model = AutoModelForCausalLM.from_pretrained(m["model_id"])
    tokenizer.save_pretrained(m["local_model_path"])
    model.save_pretrained(m["local_model_path"])

    # GGUF 변환 실행
    !python3 scripts/convert/gguf/convert.py \
      --model_path {m["local_model_path"]} \
      --outfile_path {m["gguf_output_dir"]} \
      --quantize {quant_method}

    print(f"✅ GGUF 변환 완료: {m['gguf_output_dir']}")

    # GGUF 파일 업로드
    gguf_files = glob.glob(f"{m['gguf_output_dir']}/*.gguf")
    api.create_repo(m["repo_id_gguf"], exist_ok=True)

    for filepath in gguf_files:
        filename = os.path.basename(filepath)
        upload_file(
            path_or_fileobj=filepath,
            path_in_repo=filename,
            repo_id=m["repo_id_gguf"],
            repo_type="model"
        )
        print(f"✅ 업로드 완료: {filename} → {m['repo_id_gguf']}")

    print(f"🌐 모델 업로드 완료: https://huggingface.co/{m['repo_id_gguf']}")